## SQL Advanced
https://learnsql.com/track/advanced-sql-in-postgresql/course/postgresql-window-functions/window-frame/rows/confusion

### Window Frame


For each order, show its id, the placed date, and the third column which will count the number of orders up to the current order when sorted by the placed date.
```
select
	id,
    placed,
    count(id) over(order by placed rows between unbounded preceding and current row)
from single_order
```

Warehouse workers always need to pick the products for orders by hand and one by one. For positions with order_id = 5, calculate the remaining sum of all the products to pick. For each position from that order, show its id, the id of the product, the quantity and the quantity of the remaining items (including the current row) when sorted by the id in the ascending order.
```
select
	id,
    product_id,
    quantity,
    sum(quantity) over(rows between current row and unbounded following)
from order_position
where order_id = 5
order by id asc
```

For each product, show its id, name, introduced date and the count of products introduced up to that point.
```
select
	id, name, introduced,
    count(id) over(order by introduced rows between unbounded preceding and current row)
from product
```

Now, for each single_order, show its placed date, total_price, the average price calculated by taking 2 previous orders, the current order and 2 following orders (in terms of the placed date) and the ratio of the total_price to the average price calculated as before.
```
select 
	placed, total_price,
    avg(total_price) over(order by placed rows between 2 preceding and 2 following),
    total_price::numeric / avg(total_price::numeric) over(order by placed rows between 2 preceding and 2 following)
from single_order
```

You will now have a chance to practice abbreviations. Pick those stock changes which refer to product_id = 3. For each of them, show the id, changed date, quantity, and the running total, indicating the current stock status. Sort the rows by the changed date in the ascending order.
```
select
	id, changed, quantity,
    sum(quantity) over(order by changed rows unbounded preceding)
from stock_change
where product_id = 3;
```

##### Abbreviations:
- ROWS UNBOUNDED PRECEDING means BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
- ROWS n PRECEDING means BETWEEN n PRECEDING AND CURRENT ROW
- ROWS CURRENT ROW means BETWEEN CURRENT ROW AND CURRENT ROW

For each single_order, show its placed date, total_price and the average price from the current single_order and three previous orders (in terms of the placed date).
```
select
	placed, total_price,
    avg(total_price) over(order by placed rows 3 preceding)
from single_order
```

Modify the example so that it shows the average total_price for single days for each row.
```
SELECT
  id,
  placed,
  total_price,
  AVG(total_price) OVER(ORDER BY placed range current row)
FROM single_order;
```

The difference between `ROWS` and `RANGE` is similar to the difference between the ranking functions `ROW_NUMBER` and `RANK()`

The query with `ROWS` sums the total_price for all rows which have their `ROW_NUMBER` less than or equal to the row number of the current row.
```
SELECT
  id,
  placed,
  total_price,
  ROW_NUMBER() OVER(ORDER BY placed),
  SUM(total_price) OVER(
    ORDER BY placed
    ROWS UNBOUNDED PRECEDING)
FROM single_order
```
The query with `RANGE` sums the total_price for all rows which have their `RANK()` less than or equal to the rank of the current row.
```
SELECT
  id,
  placed,
  total_price,
  RANK() OVER(ORDER BY placed),
  SUM(total_price) OVER(
    ORDER BY placed
    RANGE UNBOUNDED PRECEDING)
FROM single_order
```

For each stock_change with product_id = 7, show its id, quantity, changed date and another column to count the number of stock changes with product_id = 7 on that particular date.
```
select 
	id, quantity, changed,
    count(id) over(order by changed range current row)
from stock_change
where product_id = 7
```

For each stock_change, show id, product_id, quantity, changed date, and the total quantity change from all stock_change for that product.
```
select
	id, product_id, quantity, changed,
    sum(quantity) over(order by product_id range current row)
from stock_change
```

For each stock_change, show its id, changed date, and the number of any stock changes that took place on the same day or any time earlier.
```
select
	id, changed,
    count(id) over(order by changed range unbounded preceding)
from stock_change
```

Our finance department needs to calculate future cashflows for each date. Let's help them. In order to do that, we need to show each order: its id, placed date, total_price and the total sum of all prices of orders from the very same day or any later date.
```
select
	id, placed, total_price,
    sum(total_price) over(order by placed range between current row and unbounded following)
from single_order
```

Perfect! You may wonder what the default window frame is when it's not explicitly specified. This may differ between databases, but the most typical rule is as follows:

If you don't specify an `ORDER BY` clause within `OVER(...)`, the whole partition of rows will be used as the window frame.

If you do specify an `ORDER BY` clause within `OVER(...)`, the database will assume `RANGE UNBOUNDED PRECEDING` as the window frame. Let's check both of these cases in exercises.

Exercise
We'll start with not specifying an `ORDER BY` clause within `OVER(...)`.

For each single order, show its id, date when it was placed, the total price and the sum of all total prices.

Note that the `SUM` computes the sum of all prices in the table, even though you did not specify the window frame.

```
select
	id, placed, total_price,
    sum(total_price) over()
from single_order
```

Alright! When there is no `ORDER BY` clause in `OVER(...)`, the query simply treats all rows as the window frame for each row. Nothing shocking, really – that's the kind of queries we wrote in previous parts.

Now, we said the following: if there is an `ORDER BY` clause, `RANGE UNBOUNDED PRECEDING` will be used as the default window frame. Let's find out if it's true.

Exercise
Just as in one of the previous exercises, we'll be looking for the running sum of single orders. For each order, show its id, placed date, total_price and the sum of all total prices. Sort the orders by the placed date, but do not specify any window frame.

The sum of total_prices should be calculated as if you wrote `RANGE UNBOUNDED PRECEDING`.
```
select
	id, placed, total_price,
    sum(total_price) over(order by placed)
from single_order
```

Alright! It's time to review what we've learned in this part:

- You can define a window frame within `OVER(...)`. The syntax is: `[ROWS|RANGE] <window frame definition>`.
- `ROWS` always treats rows individually (like the `ROW_NUMBER()` function), `RANGE` also adds rows which share the same value in the column we order by (like the `RANK()` function).
- `<window frame definition>` is defined with `BETWEEN <lower bound> AND <upper bound>`, where the bounds may be defined with:
    - `UNBOUNDED PRECEDING`,
    - `n PRECEDING` (`ROWS` only),
    - `CURRENT ROW`,
    - `n FOLLOWING` (`ROWS` only),
    - `UNBOUNDED` FOLLOWING


So, are you ready for a short quiz?

For each row from department with id = 2, show its department_id, year, amount and the total amount from the current year + two previous years.
```
select
	department_id, year, amount,
    sum(amount) over(order by year rows between 2 preceding and current row)
from revenue
where department_id = 2
```

For each row from department with id = 1, show its department_id, year, amount and the running average amount from all rows up to the current rows, sorted by the year.
```
select
	department_id, year, amount,
    avg(amount) over(order by year rows unbounded preceding)
from revenue
where department_id = 1
```

For each row sorted by the year, show its department_id, year, amount, the average amount from all departments in the given year and the difference between the amount and the average amount.
```
select 
	department_id, year, amount,
    avg(amount) over(order by year range current row),
    amount - avg(amount) over(order by year range current row)
from revenue
```